In [ ]:
import numpy as np
import torch
from models.ResNet import ResNet18
from torch.utils.data import DataLoader, Dataset
from utils.ourdatasets import IM_image_datasets,GA_image_datasets,train_covid,test_covid,val_covid

In [ ]:
dataloader_kwargs = {
    'drop_last': True,
    'pin_memory': True,
    'num_workers': 4,
}
train_loader = DataLoader(IM_image_datasets['train'], batch_size=16, **dataloader_kwargs)
test_loader = DataLoader(IM_image_datasets['test'], batch_size=16, **dataloader_kwargs)
# 加载预训练模型
model_save_path = '/home/FedTC/save/IM/resnet18_iidFalse_num30_C0.1_le10_m0.9_wd0.0/shard2_sdr0.0/FedTKF_dirichlet_ablationT0/local_upt_full_aggr_full/best_model.pt'
device = 'cuda:0'
model = ResNet18().to(device)
model.load_state_dict(torch.load(model_save_path), strict=True)

# 获取训练集和测试集的尾部全连接层输出向量

train_features = []
train_labels = []
test_features = []
test_labels = []
for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(images)
        features = outputs.to('cpu').detach().numpy()
        train_features.append(features)
        train_labels.append(labels.to('cpu'))
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(images)
        features = outputs.to('cpu').detach().numpy()
        test_features.append(features)
        test_labels.append(labels.to('cpu'))
train_features = np.concatenate(train_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
test_features = np.concatenate(test_features, axis=0)
test_labels = np.concatenate(test_labels, axis=0)



In [ ]:
# 计算训练集和测试集中每个类别的尾部全连接层输出向量的均值和方差
num_classes = 10
train_means = np.zeros((num_classes, 10))
train_vars = np.zeros((num_classes, 10))
test_means = np.zeros((num_classes, 10))
test_vars = np.zeros((num_classes, 10))
for i in range(num_classes):
    train_class_features = train_features[train_labels == i]
    test_class_features = test_features[test_labels == i]
    train_means[i] = np.mean(train_class_features, axis=0)
    train_vars[i] = np.var(train_class_features, axis=0)
    test_means[i] = np.mean(test_class_features, axis=0)
    test_vars[i] = np.var(test_class_features, axis=0)
# 计算均值和方差的差异
means_diff = np.abs(train_means - test_means)
vars_diff = np.abs(train_vars - test_vars)
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
model_save_path = '/home/FedTC/save/IM/resnet18_iidFalse_num30_C0.1_le10_m0.9_wd0.0/shard2_sdr0.0/FedTKF_dirichlet_ablationT0/local_upt_body_aggr_full/best_model.pt'
device = 'cuda:0'
model = ResNet18().to(device)
model.load_state_dict(torch.load(model_save_path), strict=True)

# 获取训练集和测试集的尾部全连接层输出向量

train_features_CDL = []
train_labels_CDL = []
test_features_CDL = []
test_labels_CDL = []
for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(images)
        features = outputs.to('cpu').detach().numpy()
        train_features_CDL.append(features)
        train_labels_CDL.append(labels.to('cpu'))
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(images)
        features = outputs.to('cpu').detach().numpy()
        test_features_CDL.append(features)
        test_labels_CDL.append(labels.to('cpu'))
train_features_CDL = np.concatenate(train_features_CDL, axis=0)
train_labels_CDL = np.concatenate(train_labels_CDL, axis=0)
test_features_CDL = np.concatenate(test_features_CDL, axis=0)
test_labels_CDL = np.concatenate(test_labels_CDL, axis=0)

In [ ]:
# 计算训练集和测试集中每个类别的尾部全连接层输出向量的均值和方差
num_classes = 10
train_means = np.zeros((num_classes, 10))
train_vars = np.zeros((num_classes, 10))
test_means = np.zeros((num_classes, 10))
test_vars = np.zeros((num_classes, 10))
for i in range(num_classes):
    train_class_features = train_features_CDL[train_labels_CDL == i]
    test_class_features = test_features_CDL[test_labels_CDL == i]
    train_means[i] = np.mean(train_class_features, axis=0)
    train_vars[i] = np.var(train_class_features, axis=0)
    test_means[i] = np.mean(test_class_features, axis=0)
    test_vars[i] = np.var(test_class_features, axis=0)
# 计算均值和方差的差异
means_diff_CLD = np.abs(train_means - test_means)
vars_diff_CLD = np.abs(train_vars - test_vars)
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('pdf')
# 生成数据
x = np.array([0, 1, 2, 3, 4,5,6,7,8,9])
y1 = np.array(vars_diff.mean(axis=1))
y2 = np.array(vars_diff_CLD.mean(axis=1))

# 设置图形大小和字体
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20})

# 绘制柱状图
plt.bar(x - 0.2, y1, width=0.4, color='#FDC3BE', align='center', label='no CLD')
plt.bar(x + 0.2, y2, width=0.4, color='#018F9C', align='center', label='CLD')

# 添加图例和标签
plt.legend()
plt.xticks([0,1,2,3,4,5,6,7,8,9])
plt.xlabel('Categories')
plt.ylabel('Variance Difference')

# 显示图形
plt.show()
plt.savefig('Variance Difference.pdf')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 生成数据
x = np.array([0, 1, 2, 3, 4,5,6,7,8,9])
y1 = np.array(means_diff.mean(axis=1))
y2 = np.array(means_diff_CLD.mean(axis=1))

# 设置图形大小和字体#FE7B72

plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20})

# 绘制柱状图
plt.bar(x - 0.2, y1, width=0.4, color='#FDC3BE', align='center', label='no CLD')
plt.bar(x + 0.2, y2, width=0.4, color='#018F9C', align='center', label='CLD')

# 添加图例和标签
plt.legend()
plt.xticks([0,1,2,3,4,5,6,7,8,9])
plt.xlabel('Categories')
plt.ylabel('Mean Difference')

# 显示图形
plt.show()
plt.savefig('Mean Difference.pdf')
